In [1]:
# Define the packages
using Pkg
using CategoricalArrays
using MAT
using DataFrames
using MLJ
using LinearAlgebra
using Statistics
using Plots
using StatsPlots
using CSV

In [2]:
using Random, Statistics, ProgressMeter

"""
    permutation_test_regressor(
        X::Tables.AbstractColumns, y::AbstractVector,
        model; cv=nothing, n_perm=1000, measure=rms, rng=Random.default_rng()
    ) → (p, observed, null_dist)

Permutation test for regression.  For error metrics (lower = better) the *p*-value is
computed accordingly.
"""
function permutation_test_regressor(X, y, model;
        cv=nothing, n_perm::Int=1000,
        measure=rms, rng=Random.default_rng())

    cv === nothing && (cv = CV(nfolds=5, shuffle=true, rng=rng))
    obs_mach = machine(model, X, y)
    obs_result = evaluate!(obs_mach; resampling=cv, measures=[measure],
                           operation=predict, verbosity=0)
    observed = mean(obs_result.per_fold[1])

    null_scores = Float64[]
    @showprogress 1 "Permutation test…" for _ in 1:n_perm
        y_perm = shuffle(rng, y)
        perm_mach = machine(model, X, y_perm)
        res = evaluate!(perm_mach; resampling=cv, measures=[measure],
                        operation=predict, verbosity=0)
        push!(null_scores, mean(res.per_fold[1]))
    end
    # Lower is better, so count how many perm scores are *≤* observed
    p = (count(<=(observed), null_scores) + 1) / (n_perm + 1)

    return p, observed, null_scores
end


permutation_test_regressor

In [3]:
# sub-MOA101, sub-MOA102, sub-MOA104, sub-MOA105, sub-MOA107, sub-MOA108, sub-MOA109, sub-MOA110, sub-MOA111, sub-MOA112, sub-MOA114,
# sub-MOA115, sub-MOA116, sub-MOA118, sub-MOA121, sub-MOA122, sub-MOA123, sub-MOA124, sub-MOA126, sub-MOA127, sub-MOA128, sub-MOA130,
# sub-MOA131, sub-MOA133, sub-MOA134, sub-MOA135

# Absolute MADRS scores at session d2
y_absolute = (24,22,17,34,21,41,18,29,0,5,31,22,22,5,26,37,27,36,9,20,25,29,29,32,17,37)

# Delta change in MADRS from session b0 to session d2
y_delta = (11,9,24,2,17,-8,7,2,30,30,6,4,12,28,-1,-7,3,-1,15,12,8,9,-1,2,18,3)

(11, 9, 24, 2, 17, -8, 7, 2, 30, 30, 6, 4, 12, 28, -1, -7, 3, -1, 15, 12, 8, 9, -1, 2, 18, 3)

In [4]:
# List of subjects that have some degree of depression
target_subjects = [
    "sub-MOA101", "sub-MOA102", "sub-MOA104", "sub-MOA105", "sub-MOA107", "sub-MOA108", "sub-MOA109", "sub-MOA110", "sub-MOA111",
    "sub-MOA112", "sub-MOA114", "sub-MOA115", "sub-MOA116", "sub-MOA118", "sub-MOA121", "sub-MOA122", "sub-MOA123","sub-MOA124", 
    "sub-MOA126", "sub-MOA127", "sub-MOA128", "sub-MOA130", "sub-MOA131", "sub-MOA133", "sub-MOA134", "sub-MOA135"]

# Base path to your subject folders
base_path = "Spectral_DCM_Collection_Diag_4x4"

# Collect valid file paths
valid_files = String[]

for subj in target_subjects
    subj_path = joinpath(base_path, subj)
    ses_path = joinpath(subj_path, "ses-b0")
    glm_path = joinpath(ses_path, "glm")
    dcm_file = joinpath(glm_path, "spDCM_DMN.mat")

    if !isdir(ses_path)
        @warn "Missing session folder: $ses_path"
    elseif !isfile(dcm_file)
        @warn "Missing spDCM_DMN.mat for $subj"
    else
        push!(valid_files, dcm_file)
    end
end

println("✅ Found spDCM_DMN.mat for $(length(valid_files)) out of $(length(target_subjects)) subjects.")


✅ Found spDCM_DMN.mat for 26 out of 26 subjects.


In [5]:
# Extract A matrix features as a flat 16-element vector
function extract_features(file)
    mat = matread(file)
    A = mat["params"]  # 4×4 matrix
    return vec(Matrix(A))  # Flatten to 16-element vector
end

extract_features (generic function with 1 method)

In [6]:
# Create feature dataset
X = hcat([extract_features(file) for file in valid_files]...)'

X_df = DataFrame(X, :auto)  # convert to MLJ-compatible table

# Ensure that the number of subjects match in X_df and Y labels of different modalities
@assert size(X_df, 1) == length(y_absolute) "Mismatch between number of samples in X and y_absolute"
@assert size(X_df, 1) == length(y_delta) "Mismatch between number of samples in X and y_delta"


In [7]:
using MLJModels, MLJScikitLearnInterface, MLJBase

function evaluate_regression_model(X_df::DataFrame, y, model_label::String, nfolds::Int=5)
    # Load and chain standardizer with ElasticNetCVRegressor
    Standardizer = @load Standardizer pkg=MLJModels verbosity=0
    ElasticNetCVRegressor = @load ElasticNetCVRegressor pkg=MLJScikitLearnInterface verbosity=0

    model = Standardizer() |> ElasticNetCVRegressor()
    mach = machine(model, X_df, y)

    # Metrics
    metrics = [rms, mae]
    metric_labels = ["RMSE", "MAE"]

    # Cross-validation
    cv = CV(nfolds=nfolds, shuffle=true, rng=42)
    results = evaluate!(mach,
        resampling=cv,
        measures=metrics,
        operation=predict,
        verbosity=0
    )

    # Extract per-fold metrics
    all_scores = results.per_fold
    flat_scores = vcat(all_scores...)

    # Create DataFrame for table
    metrics_df = DataFrame(
        Fold = 1:nfolds,
        RMSE = all_scores[1],
        MAE = all_scores[2]
    )
    avg_row = DataFrame(Fold = ["Mean"], RMSE = [mean(all_scores[1])],
                        MAE = [mean(all_scores[2])])
    metrics_table = vcat(metrics_df, avg_row)

    # Save metrics table as CSV
    CSV.write("regression_metrics_4x4_$(nfolds)_fold_$(model_label).csv", metrics_table)

    # Prepare plot data
    plot_df = DataFrame(
        Fold = repeat(1:nfolds, outer=length(metrics)),
        Metric = repeat(metric_labels, inner=nfolds),
        Value = flat_scores
    )

    # Plot grouped bar
    @df plot_df groupedbar(
        string.(:Fold), :Value, group=:Metric,
        bar_position=:dodge,
        bar_width=0.2,
        xlabel="Fold", ylabel="Metric Value",
        title="Regression Metrics per Fold",
        legend=:outertop,
        guidefontsize=10,
        tickfontsize=10,
        size=(750, 500),
        dpi=300
    )
    savefig("regression_metrics_4x4_$(nfolds)_fold_$(model_label).png")
end

    CondaPkg Found dependencies: /Users/ricobenning/.julia/packages/MLJScikitLearnInterface/xHP4R/CondaPkg.toml
    CondaPkg Found dependencies: /Users/ricobenning/.julia/packages/PythonCall/L4cjh/CondaPkg.toml
    CondaPkg Initialising pixi
             │ /Users/ricobenning/.julia/artifacts/d2fecc2a9fa3eac2108d3e4d9d155e6ff5dfd0b2/bin/pixi
             │ init
             │ --format pixi
             └ /Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/.CondaPkg
✔ Created /Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/.CondaPkg/pixi.toml
    CondaPkg Wrote /Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/.CondaPkg/pixi.toml
             │ [dependencies]
             │ openssl = ">=3, <3.6"
             │ 
             │     [dependencies.scikit-learn]
             │     channel = "conda-forge"
        

evaluate_regression_model (generic function with 2 methods)

In [8]:
evaluate_regression_model(X_df, collect(y_absolute), "elasticnet_cv_absolute")

"/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/regression_metrics_4x4_5_fold_elasticnet_cv_absolute.png"

In [9]:
evaluate_regression_model(X_df, collect(y_delta), "elasticnet_cv_delta")

"/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/regression_metrics_4x4_5_fold_elasticnet_cv_delta.png"

In [10]:
# Permutation test call
# regression – delta MADRS
ElasticNetRegressor = @load ElasticNetRegressor pkg=MLJLinearModels
p_r, rmse, null_rmse = permutation_test_regressor(X_df, collect(y_delta),
                                                  ElasticNetRegressor(); n_perm=500)
println("Permutation p-value (RMSE) = $p_r")

import MLJLinearModels ✔

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159


┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc MLJLinearModels.ElasticNetRegressor` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{AbstractVector{Continuous}}, AbstractVector{Count}}
│ 
│ fit_data_scitype(model) = Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{Continuous}}
└ @ MLJBase /Users/ricobenning/.julia/packages/MLJBase/7nGJF/src/machines.jl:237
┌ Warning: No appropriate st

In [11]:
Pkg.add("MLUtils")
Pkg.instantiate()

In [12]:
using Random
using MLUtils

@load ElasticNetRegressor pkg=MLJLinearModels
model = ElasticNetRegressor()

# Prepare 5-fold CV
nfolds = 5
folds = kfolds(eachindex(y_absolute), k=nfolds)

y_absolute = collect(y_absolute)

# Loop over folds
all_preds = DataFrame()
for (i, (train_indices, test_indices)) in enumerate(folds)
    # Select training data
    X_train = X_df[train_indices, :] # Select all columns for the training rows
    y_train = y_absolute[train_indices]

    # Select testing data
    X_test = X_df[test_indices, :] # Select all columns for the testing rows
    y_test = y_absolute[test_indices]

    # Train model
    mach = machine(model, X_train, collect(y_train))
    fit!(mach)
    
    # Ensure predictions are Float64 scalars
    y_pred = predict(mach, X_test)
    y_pred = [pred[1] for pred in y_pred]

    # Sort by test sample index to align curves (or use any logical sort key)
    sorted_indices = sortperm(test_indices)
    y_test_sorted = y_test[sorted_indices]
    y_pred_sorted = y_pred[sorted_indices]

    # Plot true vs predicted as connected curves
    p = plot(
        y_test_sorted,
        label = "True",
        lw = 2,
        xlabel = "Sample Index",
        ylabel = "Value",
        title = "ElasticNet Regression on Absolute Values_4x4: Fold $i",
        size=(750, 500),
        dpi=300
    )
    plot!(p, y_pred_sorted, label = "Predicted", lw = 2, color = :orange)

    # Scatter dots on top of curves
    scatter!(p, y_test_sorted, label = "", marker = (:circle, 4), color = :green)
    scatter!(p, y_pred_sorted, label = "", marker = (:diamond, 4), color = :red)

    # Save plot
    savefig(p, "elasticnet_fold_$(i)_pred_vs_true_absolute_4x4.png")
end

In [13]:
@load ElasticNetRegressor pkg=MLJLinearModels
model = ElasticNetRegressor()

# Prepare 5-fold CV
nfolds = 5
folds = kfolds(eachindex(y_absolute), k=nfolds)

y_delta = collect(y_delta)

# Loop over folds
all_preds = DataFrame()
for (i, (train_indices, test_indices)) in enumerate(folds)
    # Select training data
    X_train = X_df[train_indices, :] # Select all columns for the training rows
    y_train = y_delta[train_indices]

    # Select testing data
    X_test = X_df[test_indices, :] # Select all columns for the testing rows
    y_test = y_delta[test_indices]

    # Train model
    mach = machine(model, X_train, collect(y_train))
    fit!(mach)
    
    # Ensure predictions are Float64 scalars
    y_pred = predict(mach, X_test)
    y_pred = [pred[1] for pred in y_pred]

    # Sort by test sample index to align curves (or use any logical sort key)
    sorted_indices = sortperm(test_indices)
    y_test_sorted = y_test[sorted_indices]
    y_pred_sorted = y_pred[sorted_indices]

    # Plot true vs predicted as connected curves
    p = plot(
        y_test_sorted,
        label = "True",
        lw = 2,
        xlabel = "Sample Index",
        ylabel = "Value",
        title = "ElasticNet Regression on Delta Values_4x4: Fold $i",
        size=(750, 500),
        dpi=300
    )
    plot!(p, y_pred_sorted, label = "Predicted", lw = 2, color = :orange)

    # Scatter dots on top of curves
    scatter!(p, y_test_sorted, label = "", marker = (:circle, 4), color = :green)
    scatter!(p, y_pred_sorted, label = "", marker = (:diamond, 4), color = :red)

    # Save plot
    savefig(p, "elasticnet_fold_$(i)_pred_vs_true_delta_4x4.png")
end